In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import random

# Carregando os dados

Praticamente todos os dados são categóricos, portanto algum método de vetorização
deve ser necessário.

In [2]:
df = pd.read_excel('../data/sales.xlsx', engine='openpyxl')

In [3]:
df.head()

,YEAR,WEEK,PRODUCT_ID,SECTOR1_ID,SECTOR2_ID,SECTOR3_ID,SECTOR4_ID,SALES_QUANTITY
0,2017,7,1,1,1,1,1,14.00
1,2015,27,2,1,2,2,2,41.00
2,2015,24,3,2,3,3,3,10.46
3,2016,36,4,3,4,4,4,46.00
4,2016,6,5,4,5,5,5,5.00


In [4]:
df.describe()

,YEAR,WEEK,PRODUCT_ID,SECTOR1_ID,SECTOR2_ID,SECTOR3_ID,SECTOR4_ID,SALES_QUANTITY
count,442573.000000,442573.000000,442573.000000,442573.000000,442573.000000,442573.000000,442573.000000,442573.000000
mean,2015.800880,27.532669,3481.469276,2.415721,5.473027,26.874136,61.164072,17.953565
std,0.632561,15.512948,2202.165928,1.363241,4.058683,21.175039,49.865031,58.916102
min,2015.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.005000
25%,2015.000000,12.000000,1615.000000,1.000000,1.000000,10.000000,18.000000,2.000000
50%,2016.000000,29.000000,3282.000000,2.000000,6.000000,20.000000,48.000000,6.000000
75%,2016.000000,41.000000,5157.000000,4.000000,8.000000,39.000000,97.000000,14.000000
max,2017.000000,52.000000,9372.000000,7.000000,22.000000,104.000000,222.000000,1891.265000


# Separando treino, validação e teste

Os dados de teste será os da semana 11 do ano 2017, o resto dos dados será dividido
aleatóriamente em treino e validão na proporção 80/20.

In [18]:
all_indexes = set(df.index)
test_indexes = set(df.loc[(df['YEAR'] == 2017) & (df['WEEK'] == 11)].index)
remaining_indexes = all_indexes - test_indexes

random.seed(42)
val_indexes = set(random.sample(remaining_indexes, 
    int(len(remaining_indexes)*0.20)))

train_indexes = remaining_indexes - val_indexes

In [25]:
#sanity check
print(len(all_indexes), len(test_indexes), len(train_indexes), len(val_indexes))
print(train_indexes.isdisjoint(test_indexes))
print(train_indexes.isdisjoint(val_indexes))
print(test_indexes.isdisjoint(val_indexes))

442573 4891 350146 87536
True
True
True


In [52]:
train_data = df.filter(items = list(train_indexes), axis=0)
val_data = df.filter(items = list(val_indexes), axis=0)
test_data = df.filter(items = list(test_indexes), axis=0)

In [53]:
y_train = train_data['SALES_QUANTITY']
y_val = val_data['SALES_QUANTITY']
y_test = test_data['SALES_QUANTITY']

In [54]:
X_train = train_data[train_data.columns[:-1]]
X_val = val_data[val_data.columns[:-1]]
X_test = test_data[test_data.columns[:-1]]

# Modelo Baseline

Vamos treinar um modelo random forest sem nenhuma transformação nos dados para 
ver o que acontece

In [45]:
rf = RandomForestRegressor(max_depth=10, random_state=42)
rf.fit(np.array(X_train), np.array(y_train))

RandomForestRegressor(max_depth=10, random_state=42)

In [57]:
print(np.sqrt(mean_squared_error(np.array(y_test), 
    rf.predict(np.array(X_test)))))

print(mean_absolute_error(np.array(y_test), rf.predict(np.array(X_test))))

print('\n')

print(np.sqrt(mean_squared_error(np.array(y_val), 
    rf.predict(np.array(X_val)))))

print(mean_absolute_error(np.array(y_val), rf.predict(np.array(X_val))))

32.06549944565198
12.530874160613662


32.59756814812514
12.555507610129522


# One-hot encoding para lidar com as variáveis categóricas

Como as classes são muitas (~10k), a dimensionalidade do problema aumentará
exponencialmente.

Por isso vamos usar também o método para representações esparsas.

In [48]:
def encode_df(df, columns):
    labeler = LabelEncoder()
    onehot = OneHotEncoder(sparse=True)

    empty_array = np.empty((len(df), 0), int)
    for column in columns:
        numeric_data = labeler.fit_transform(df[column])
        numeric_data = numeric_data.reshape(len(df), 1)
        empty_array = np.append(empty_array, numeric_data, 1)

        data = onehot.fit_transform(empty_array)

    return data

In [49]:
x = encode_df(df.copy(), list(df.columns)[:-1])

In [68]:
hotrf = RandomForestRegressor(max_depth=10, random_state=42)
hotrf.fit(x[list(train_indexes)], np.array(y_train))

RandomForestRegressor(max_depth=10, random_state=42)

In [70]:
print(np.sqrt(mean_squared_error(np.array(y_test), 
    hotrf.predict(x[list(test_indexes)]))))

print(mean_absolute_error(np.array(y_test), 
    hotrf.predict(x[list(test_indexes)])))

print('\n')

print(np.sqrt(mean_squared_error(np.array(y_val), 
    hotrf.predict(x[list(val_indexes)]))))

print(mean_absolute_error(np.array(y_val), 
    hotrf.predict(x[list(val_indexes)])))

34.90881422696889
15.52213065327469


37.18737867843924
15.542937799229975


É possível notar uma piora no resultado. Deve ser investigada, talvez o modelo
esparso precise de mais iterações, talvez a esparsidade piore o problema.

Testamos também a redução de dimensionalidade porém ficará omitida aqui pois 
o algoritmo do RandomForest não convergiu com o critério de parada estabelecido.

Vamos agora testar o LightGBM

# LightGBM

Vamos testar o LGBM primeiramente sem nenhuma transformação nos dados.

In [74]:
lgbm_train = lgb.Dataset(np.array(X_train), label=np.array(y_train))
lgbm_val = lgb.Dataset(np.array(X_val), 
    reference=lgbm_train, 
    label=np.array(y_val))

param = {'num_leaves': 31, 'objective': 'regression'}
param['metric'] = 'RMSE'

num_round = 50
bst = lgb.train(param, lgbm_train, num_round, valid_sets=[lgbm_val])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 667
[LightGBM] [Info] Number of data points in the train set: 350146, number of used features: 7
[LightGBM] [Info] Start training from score 18.060209
[1]	valid_0's rmse: 54.9403
[2]	valid_0's rmse: 53.1277
[3]	valid_0's rmse: 51.5291
[4]	valid_0's rmse: 50.1452
[5]	valid_0's rmse: 49.115
[6]	valid_0's rmse: 48.0521
[7]	valid_0's rmse: 47.2446
[8]	valid_0's rmse: 46.6118
[9]	valid_0's rmse: 45.9305
[10]	valid_0's rmse: 45.3494
[11]	valid_0's rmse: 44.8644
[12]	valid_0's rmse: 44.4052
[13]	valid_0's rmse: 43.962
[14]	valid_0's rmse: 43.4732
[15]	valid_0's rmse: 43.1078
[16]	valid_0's rmse: 42.7598
[17]	valid_0's rmse: 42.54
[18]	valid_0's rmse: 42.2355
[19]	valid_0's rmse: 41.9229
[20]	valid_0's rmse: 41.6698
[21]	valid_0's rmse: 41.

In [79]:
ypred = bst.predict(np.array(X_test))

print(np.sqrt(mean_squared_error(np.array(y_test), np.array(ypred))))

print(mean_absolute_error(np.array(y_test), np.array(ypred)))

#Ainda abaixo do baseline

36.59249450747933
13.967531591180608


# Redução de dimensionalidade com SVD

Vamos fazer one-hot encoding dos dados e aplicar Truncated SVD nos dados 
para reduzir as dimensões do problema.

Seria interessante podermos testar também tSNE ou PCA, porém a esparsidade 
não nos permite.

In [80]:
#Vamos usar 1000 componentes, o que representa uma redução de cerca de 90%.

svd = TruncatedSVD(n_components=1000, n_iter=5, random_state=42)
svd.fit(x[list(train_indexes)])

TruncatedSVD(n_components=1000, random_state=42)

In [81]:
#Reduzimos a dimensão do problema em cerca de 90% e ainda temos 86% de sua 
#representatividade
print(svd.explained_variance_ratio_.sum())

0.8644461818045971


In [86]:
lgbm_train = lgb.Dataset(svd.transform(x[list(train_indexes)]),
    label=np.array(y_train))

lgbm_val = lgb.Dataset(svd.transform((x[list(val_indexes)])), 
    reference=lgbm_train, 
    label=np.array(y_val))

param = {'num_leaves': 31, 'objective': 'regression'}
param['metric'] = 'RMSE'

num_round = 50
bst = lgb.train(param, lgbm_train, num_round, valid_sets=[lgbm_val])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.420694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255000
[LightGBM] [Info] Number of data points in the train set: 350146, number of used features: 1000
[LightGBM] [Info] Start training from score 18.060209
[1]	valid_0's rmse: 53.1767
[2]	valid_0's rmse: 49.7245
[3]	valid_0's rmse: 46.6791
[4]	valid_0's rmse: 43.9142
[5]	valid_0's rmse: 41.603
[6]	valid_0's rmse: 39.5703
[7]	valid_0's rmse: 37.8434
[8]	valid_0's rmse: 36.3044
[9]	valid_0's rmse: 34.9241
[10]	valid_0's rmse: 33.6658
[11]	valid_0's rmse: 32.6059
[12]	valid_0's rmse: 31.6981
[13]	valid_0's rmse: 30.8577
[14]	valid_0's rmse: 30.1428
[15]	valid_0's rmse: 29.5226
[16]	valid_0's rmse: 29.0099
[17]	valid_0's rmse: 28.5402
[18]	valid_0's rmse: 28.1198
[19]	valid_0's rmse: 27.7452
[20]	valid_0's rmse: 27.4322
[21]	valid_0's rmse: 27.1165
[22]	valid_0's rmse: 26.8438
[23]	valid_0's rmse: 

In [88]:
ypred = bst.predict(svd.transform(x[list(test_indexes)]))

print(np.sqrt(mean_squared_error(np.array(y_test), np.array(ypred))))

print(mean_absolute_error(np.array(y_test), np.array(ypred)))

#Agora sim houve uma melhora tanto no RMSE quanto no MAE.

25.016153433833722
10.496038888902202


In [97]:
lgbm_train = lgb.Dataset(np.array(X_train), label=np.array(y_train))
lgbm_val = lgb.Dataset(np.array(X_val), 
    reference=lgbm_train, 
    label=np.array(y_val))

param = {'num_leaves': 31, 'objective': 'regression'}
param['metric'] = 'RMSE'

num_round = 50
bst = lgb.train(param, lgbm_train, num_round, valid_sets=[lgbm_val], 
    categorical_feature=[item for item in range(len(X_train.columns))])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8268
[LightGBM] [Info] Number of data points in the train set: 350146, number of used features: 7
[LightGBM] [Info] Start training from score 18.060209
[1]	valid_0's rmse: 52.4883
[2]	valid_0's rmse: 48.289
[3]	valid_0's rmse: 44.5368
[4]	valid_0's rmse: 41.2152
[5]	valid_0's rmse: 38.3708
[6]	valid_0's rmse: 35.8971
[7]	valid_0's rmse: 33.6755
[8]	valid_0's rmse: 31.8108
[9]	valid_0's rmse: 30.2171
[10]	valid_0's rmse: 28.85
[11]	valid_0's rmse: 27.7067
[12]	valid_0's rmse: 26.7079
[13]	valid_0's rmse: 25.8291
[14]	valid_0's rmse: 25.0873
[15]	valid_0's rmse: 24.4561
[16]	valid_0's rmse: 23.9068
[17]	valid_0's rmse: 23.4007
[18]	valid_0's rmse: 22.9974
[19]	valid_0's rmse: 22.6672
[20]	valid_0's rmse: 22.3681
[21]	valid_0's rmse: 22.1448
[22]	valid_0's rmse: 21.945
[23]	valid_0's rmse: 21.7686


In [98]:
ypred = bst.predict(np.array(X_test))

print(np.sqrt(mean_squared_error(np.array(y_test), np.array(ypred))))

print(mean_absolute_error(np.array(y_test), np.array(ypred)))

#Melhor modelo encontrado até então, inclusive em questão de tempo de 
#implementação e treino.

19.163811687037494
7.54099396515019


# Próximos Passos

Tomar como base o melhor modelo e fazer uma busca dos hiperparâmetros que
otimizem o problema.

Além disso, previsão de demanda parece ser um bom problema para ser resolvido 
com séries temporais (essas não tenho familiaridade até então) ou talvez com
RNNs.